In [1]:
import os
import sys
import multiprocessing
import pandas as pd
import matplotlib.pyplot as plt
plt.switch_backend('TkAgg')
import time
import csv
import re

from tqdm import tqdm
import matplotlib.cm as cm
import numpy as np
from PIL import Image
import glob
import requests
from xml.etree import ElementTree
import folium
from folium.plugins import MarkerCluster


/home/an/anaconda2/lib/python2.7/site-packages/folium/__init__.py:59: UserWarning: This version of folium is the last to support Python 2. Transition to Python 3 to be able to receive updates and fixes. Check out https://python3statement.org/ for more info.
  UserWarning


In [2]:
filepath = "/home/an/work/cyngn/data/octant.io/csv/"
os.chdir(filepath)
df = pd.read_csv("combined_csv.csv") 


In [3]:
# function to convert degrees minutes seconds coordinates (DMS) to decimal degrees
def dms2dd(s):
    # example: s = """## deg ##' ##.##" L"""
    degrees, deg, minutes, seconds, direction = re.split(r'[\s-]+', s)
    dd = float(degrees) + float(minutes[:len(minutes)-1])/60 + float(seconds[:len(seconds)-1])/(60*60);
    if direction in ('S','W'):
        dd*= -1
    return dd

gps_df = df[['gps_latitude', 'gps_longitude']].copy()

gps_df['gps_latitude'] = gps_df['gps_latitude'].apply(dms2dd)
gps_df['gps_longitude'] = gps_df['gps_longitude'].apply(dms2dd)


In [4]:
latitudes = gps_df["gps_latitude"].tolist()
longitudes = gps_df["gps_longitude"].tolist()
count = 0


map = folium.Map(location=[latitudes[0], longitudes[0]], tiles='CartoDB dark_matter', zoom_start=12)
marker_cluster = MarkerCluster().add_to(map)

for i in range(0, len(latitudes), 21):
    count += 1
    folium.Marker([latitudes[i], longitudes[i]]).add_to(marker_cluster)
map